### Train with LLama index on customized data
- https://github.com/run-llama/finetune-embedding/blob/main/finetune.ipynb

In [1]:
import os, sys
sys.path.insert(0,'../../libs')

import uuid
import pandas as pd
from datasets import Dataset,load_dataset,concatenate_datasets,load_from_disk
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer
from sentence_transformers import losses
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses

import wandb

run = wandb.init(
    # Set the project where this run will be logged
    project="sentence_bert_test",
    # config={
    #     "learning_rate": 0.01,
    #     "epochs": 10,
    # }
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chuang16. Use `wandb login --relogin` to force relogin


In [2]:
raw_data_path = '/home/chengyu.huang/project/Fund_projects/all_qa_pairs_2020_01_22.xlsx'
df = pd.read_excel(raw_data_path)
df = df[['question','context']]
df.columns= ['query','context']
df.head()

,query,context
0,What was the impact of the global financia cri...,REPUBLIC OF SAN MARINO STAFF REPORT FOR THE 20...
1,What are the effects of a weak banking system ...,REPUBLIC OF SAN MARINO STAFF REPORT FOR THE 20...
2,What is the main challenge facing an economy t...,REPUBLIC OF SAN MARINO STAFF REPORT FOR THE 20...
3,Under which conditions the financial system is...,REPUBLIC OF SAN MARINO STAFF REPORT FOR THE 20...
4,What policy mix could allow the authorities to...,UKRAINE 2016 ARTICLE IV CONSULTATION AND THIRD...


In [3]:
# Randomly sample 70% of the data
df_train = df.sample(frac=0.7, random_state=42)
# Get the remaining 30% of the data
df_eval = df.drop(df_train.index)

#### create train data loader for training

In [4]:
examples = []
for index,row in df_train.iterrows():
   example = InputExample(texts=[row['query'],row['context']])
   examples.append(example)
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)                              

#### Create evaluate dataset for eval

In [5]:
## generate id entirely randomly
queries = {}
relevant_docs = {}
corpus = {}
context_groups = df_eval.groupby('context')
for group_name,group_df in context_groups:
    corpus_id = str(uuid.uuid4())
    corpus[corpus_id] = group_df['context'].iloc[0]
    for index,row in group_df.iterrows():
        question_id = str(uuid.uuid4())
        queries[question_id] = row['query']
        relevant_docs[question_id] = [corpus_id]
        
evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [6]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [7]:
### here we only have paired data, use multiple negative loss 
#train_loss = losses.TripletLoss(model=model)
loss = losses.MultipleNegativesRankingLoss(model)

In [9]:
EPOCHS = 20
warmup_steps = int(len(train_dataloader) * EPOCHS * 0.1)

## the step out from sbert is so stupid 
def callback_wb(score,epoch,steps):
    if steps < 0:
        steps = epoch*len(train_dataloader)
    else:
        steps = epoch*len(train_dataloader) + steps
    wandb.log({'acc':score,'epoch':epoch,'steps':steps})
    return None

model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluator=evaluator, 
    evaluation_steps=20,
    callback=callback_wb
)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]